<a href="https://colab.research.google.com/github/ethanknights/tune-LLM/blob/main/tune_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch transformers pandas

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [2]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch
import logging

logging.basicConfig(level=logging.INFO)


In [3]:

def generate_dataset():
    dataset = [
        ("There's a large pothole on Main Street.", "Pothole"),
        ("I hit a pothole on Elm Avenue and my tire popped.", "Pothole"),
        ("There's a deep pothole near the intersection of Oak and Pine streets.", "Pothole"),
        ("There's graffiti on the park wall.", "Graffiti"),
        ("Someone spray-painted graffiti on the bus stop.", "Graffiti"),
        ("There's offensive graffiti on the side of the building downtown.", "Graffiti"),
        ("There's loud construction noise coming from the building next door.", "Noise Complaint"),
        ("My neighbor's party is too loud and it's keeping me awake.", "Noise Complaint"),
        ("The neighbors are playing loud music late at night.", "Noise Complaint"),
        ("The trash bins haven't been emptied for days.", "Trash Pickup Request"),
        ("There's trash scattered all over the sidewalk.", "Trash Pickup Request"),
        ("The garbage truck missed our street on pickup day.", "Trash Pickup Request"),
        ("The street light at the corner of Maple and Elm streets is out.", "Street Light Outage"),
        ("There's a dark area on the street because the light isn't working.", "Street Light Outage"),
        ("The street light flickers on and off intermittently.", "Street Light Outage"),
    ]

    train_size = int(0.8 * len(dataset))
    train_dataset = dataset[:train_size]
    valid_dataset = dataset[train_size:]

    return train_dataset, valid_dataset


In [4]:

train_dataset, valid_dataset = generate_dataset()

train_df = pd.DataFrame(train_dataset, columns=["text", "label"])
valid_df = pd.DataFrame(valid_dataset, columns=["text", "label"])


In [5]:

class CustomDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 0]
        label = self.df.iloc[idx, 1]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_attention_mask=True,
            return_tensors='pt',
            padding='max_length',
            truncation=True
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(0)  # Placeholder label
        }




In [ ]:
tokenizer = AutoTokenizer.from_pretrained("unsloth/llama-3-8b-Instruct")
model = AutoModelForCausalLM.from_pretrained("unsloth/llama-3-8b-Instruct")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_dir = '/content/drive/My Drive/tune-LLM-results'


In [ ]:

train_dataset = CustomDataset(train_df, tokenizer, max_length=512)
valid_dataset = CustomDataset(valid_df, tokenizer, max_length=512)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    eval_steps=500,  # Run evaluation every 500 steps
    evaluation_strategy='epoch'
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = (preds == labels).sum().item() / len(labels)
    return {"accuracy": accuracy}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics
)



In [ ]:
logging.info("Starting training...")
trainer.train()
logging.info("Training completed.")